In [ ]:
import grpc


from reachy_sdk_api import joint_pb2_grpc
from reachy_sdk_api.joint_pb2 import JointField, JointId, JointsStateRequest
from google.protobuf.wrappers_pb2 import BoolValue, FloatValue

channel = grpc.insecure_channel(f'192.168.86.33:50055')
state_stub = joint_pb2_grpc.JointServiceStub(channel)

In [ ]:
import time
import numpy as np

goal_pos = []
present_pos = []
time_array = []
alive = True

allbody = [
        JointId(name='l_shoulder_pitch'),
        JointId(name='l_shoulder_roll'),
        JointId(name='l_arm_yaw'),
        JointId(name='l_elbow_pitch'),
        JointId(name='l_forearm_yaw'),
        JointId(name='l_wrist_pitch'),
        JointId(name='l_wrist_roll'),
        JointId(name='l_gripper'),
        JointId(name='r_shoulder_pitch'),
        JointId(name='r_shoulder_roll'),
        JointId(name='r_arm_yaw'),
        JointId(name='r_elbow_pitch'),
        JointId(name='r_forearm_yaw'),
        JointId(name='r_wrist_pitch'),
        JointId(name='r_wrist_roll'),
        JointId(name='r_gripper'),
        JointId(name='neck_disk_top'),
        JointId(name='neck_disk_middle'),
        JointId(name='neck_disk_bottom'),
    ]

In [ ]:
def get_pos():
    state_stub = joint_pb2_grpc.JointServiceStub(channel)
    
    i=0
    duration = 0
    pres_pos = JointsStateRequest(
        ids=allbody,
        requested_fields=[JointField.GOAL_POSITION, JointField.PRESENT_POSITION]
    )

    #while alive:
    while(duration < 10):
        tic = time.time()
        single_present_pos = []
        single_goal_pos = []

        state = state_stub.GetJointsState(pres_pos)

        for motor in state.states:
            x = motor.goal_position.value
            single_goal_pos.append(x)
            y = motor.present_position.value
            single_present_pos.append(y)

        goal_pos.append(np.array(single_goal_pos))
        present_pos.append(np.array(single_present_pos))
        time_array.append(time.time())
        # print(time.time() - tic)
        time.sleep(0.01)
        duration += time.time() - tic
    #print(duration)
    print("stop")
        
        
from threading import Thread

t = Thread(target=get_pos)
time.sleep(5)
print("start")
t.start()

In [ ]:
alive = False
t.join()

In [ ]:
present_pos

In [ ]:
goal_pos

In [ ]:
len(present_pos)

In [ ]:
np.save('present_pos.npy', np.array(present_pos))

In [ ]:
np.save('goal_pos.npy', np.array(goal_pos))

In [ ]:
np.save('time_array.npy', np.array(time_array))

## Replay

In [ ]:
import grpc

from reachy_sdk_api import joint_pb2, joint_pb2_grpc
from joint_pb2 import JointsCommand, JointCommand
import numpy as np
from google.protobuf.wrappers_pb2 import BoolValue, FloatValue

channel = grpc.insecure_channel(f'192.168.86.33:50055')

In [ ]:
present_pos = np.load('present_pos.npy')
goal_pos = np.load('goal_pos.npy')
send_alive = True

In [ ]:
all_ids = np.array([allbody for i in range(len(goal_pos))])

In [ ]:
import time
import numpy as np

send_alive = True

cmd_stub = joint_pb2_grpc.JointServiceStub(channel)

In [ ]:
compliancy = False

request = JointsCommand(
            commands=[
                JointCommand(id=name, compliant=BoolValue(value=compliancy))
                for name in allbody
            ]
        )
cmd_stub.SendJointsCommands(request)

send_alive = True

In [ ]:
### GOAL_POSITION

def send_pos():
    i = 0
    tic = time.time()
    while ((i < len(goal_pos)) and (send_alive)):
        toc = time.time()
        command = JointsCommand(
        commands=[
                JointCommand(id=name, goal_position=FloatValue(value=pos))
                for name, pos in zip(all_ids[i], goal_pos[i])
            ]
        )

        cmd_stub.SendJointsCommands(command)
        #print(time.time()-toc)
        i += 1
        time.sleep(0.01)

        
from threading import Thread

t2 = Thread(target=send_pos)
t2.start()

In [ ]:
### PRESENT_POSITION

def send_pos():
    i = 0
    tic = time.time()
    while ((i < len(goal_pos)) and (send_alive)):
        toc = time.time()
        command = JointsCommand(
        commands=[
                JointCommand(id=name, goal_position=FloatValue(value=pos))
                for name, pos in zip(all_ids[i], present_pos[i])
            ]
        )

        cmd_stub.SendJointsCommands(command)
        #print(time.time()-toc)
        i += 1
        time.sleep(0.01)

        
from threading import Thread

t2 = Thread(target=send_pos)
t2.start()

In [ ]:
send_alive = False
t2.join()

In [ ]:
compliancy = True

request = JointsCommand(
    commands=[
        JointCommand(id=name, compliant=BoolValue(value=compliancy))
        for name in allbody
    ]
)

cmd_stub.SendJointsCommands(request)

In [ ]:
i=3
for name, pos in zip(all_ids[i], goal_pos[i]):
    print(name, pos)

In [ ]:
allbody

In [ ]:
len(goal_pos)

In [ ]:
all_ids